# Tarea 2: Unit Commitment
## Pruebas

In [1]:
using DataFrames, XLSX
using JuMP , HiGHS      # https://jump.dev/JuMP.jl/stable/manual/models/ 
using Gurobi

Leer archivo excel

In [2]:
xf = XLSX.readxlsx("Case014.XLSX")

XLSXFile("Case014.XLSX") containing 5 Worksheets
            sheetname size          range        
-------------------------------------------------
                Buses 16x5          A1:E16       
               Demand 19x51         A1:AY19      
           Generators 11x20         A1:T11       
                Lines 22x7          A1:G22       
           Renewables 30x25         A1:Y30       


Nombres de cada hoja

In [39]:
XLSX.sheetnames(xf)[1:5]

5-element Vector{String}:
 "Buses"
 "Demand"
 "Generators"
 "Lines"
 "Renewables"

In [40]:
XLSX.sheetnames(xf)[2]

"Demand"

In [41]:
length(XLSX.sheetnames(xf))

5

Guardar hojas en lugares distintos

In [3]:
buses = xf["Buses"]
demanda = xf["Demand"]
generadores = xf["Generators"]
lineas = xf["Lines"]
renovables = xf["Renewables"] 

30×25 XLSX.Worksheet: ["Renewables"](A1:Y30) 

Acceder a datos

In [27]:
println(renovables["B3"])   # B3
println(renovables[3,2])    # fila3 , columna2(B)

31.167028084846212
31.167028084846212


In [28]:
renovables["A2:D4"]

3×4 Matrix{Any}:
 "Gen/Hour"   1       2        3
 "Wind2"     31.167  29.4256  27.6723
 "Solar8"     0.0     0.0      0.0

In [29]:
df3 = DataFrame(renovables["A2:D4"])

ArgumentError: ArgumentError: `DataFrame` constructor from a `Matrix` requires passing :auto as a second argument to automatically generate column names: `DataFrame(matrix, :auto)`

Funcion de detencion (en "END")

In [4]:
# Definir la función de parada
function stop_condition(row)
    return isempty(row[1]) || row[1] == "END"
end

stop_condition (generic function with 1 method)

### Leer tablas de hojas de datos

Buses

In [5]:
df1 = DataFrame(XLSX.gettable(buses,"A:B",stop_in_row_function=stop_condition))

Row,Bus,Vmax [pu]
,Any,Any
1,1,1.06
2,2,1.06
3,3,1.06
4,4,1.06
5,5,1.06
6,6,1.06
7,7,1.06
8,8,1.06
9,9,1.06


Caso de demanda reactiva (Q)

In [6]:
df2 = DataFrame(XLSX.gettable(demanda,"AA:AY",first_row=2,stop_in_row_function=stop_condition))

Row,Bus/Hour,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,Bus1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Bus2,12.7,10.3561,9.10085,8.15938,7.84556,9.41467,10.9838,12.2391,12.8667,13.8082,13.9651,13.1805,12.5529,11.9253,13.8082,14.122,13.3375,13.9651,14.7497,15.3773,15.6911,14.122,13.6513,12.8667
3,Bus3,19.0,15.4934,13.6154,12.2069,11.7375,14.0849,16.4324,18.3104,19.2494,20.6579,20.8927,19.7189,18.7799,17.8409,20.6579,21.1274,19.9537,20.8927,22.0664,23.0054,23.4749,21.1274,20.4232,19.2494
4,Bus4,-3.9,-3.18023,-2.79475,-2.50564,-2.40927,-2.89112,-3.37297,-3.75846,-3.9512,-4.24031,-4.28849,-4.04757,-3.85483,-3.66208,-4.24031,-4.33668,-4.09575,-4.28849,-4.52942,-4.72216,-4.81853,-4.33668,-4.19212,-3.9512
5,Bus5,1.6,1.30471,1.14656,1.02795,0.988417,1.1861,1.38378,1.54193,1.621,1.73961,1.75938,1.66054,1.58147,1.50239,1.73961,1.77915,1.68031,1.75938,1.85822,1.9373,1.97683,1.77915,1.71985,1.621
6,Bus6,7.5,6.11583,5.37452,4.81853,4.6332,5.55985,6.48649,7.2278,7.59846,8.15444,8.2471,7.78378,7.41313,7.04247,8.15444,8.33977,7.87645,8.2471,8.71042,9.08108,9.26641,8.33977,8.06178,7.59846
7,Bus7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Bus8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Bus9,16.6,13.5364,11.8956,10.665,10.2548,12.3058,14.3568,15.9975,16.8179,18.0485,18.2536,17.2281,16.4077,15.5873,18.0485,18.4587,17.4332,18.2536,19.2791,20.0995,20.5097,18.4587,17.8434,16.8179


In [10]:
size(df2)

(14, 25)

generadores

In [9]:
df3 = DataFrame(XLSX.gettable(generadores,"A:T",first_row=1,stop_in_row_function=stop_condition))

Row,Generator,Bus,Pmax [MW],Pmin [MW],Qmax [MVAR],Qmin [MVAR],Ramp [MW/h],SRamp [MW],MinUP,MinDW,InitS,InitP,StartUpCost [$],FixedCost [$],VariableCost [$/MWh],Type,PminFactor,QFactor,RampFactor,StartUpCostFactor
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,G1,1,100,40,70,-70,20,40.01,8,8,-8,0,3000,600,20,Coal,0.4,0.7,0.2,30
2,G2,2,75,7.5,52.5,-52.5,22.5,22.5,4,4,-4,0,2250,450,40,Gas,0.1,0.7,0.3,30
3,G3,3,75,7.5,52.5,-52.5,22.5,22.5,4,4,-4,0,2250,450,60,Gas,0.1,0.7,0.3,30
4,G6,6,75,7.5,52.5,-52.5,22.5,22.5,4,4,-4,0,2250,450,80,Gas,0.1,0.7,0.3,30
5,G8,8,75,7.5,52.5,-52.5,22.5,22.5,4,4,-4,0,2250,450,100,Gas,0.1,0.7,0.3,30
6,Wind2,2,34.3074,0,24.0152,-24.0152,343.074,343.074,1,1,-1,0,0,0,0,Wind,0.0,0.7,10.0,0
7,Solar8,8,36.0918,0,25.2643,-25.2643,360.918,360.918,1,1,-1,0,0,0,0,Solar,0.0,0.7,10.0,0


Renovables

In [106]:
df4 = DataFrame(XLSX.gettable(renovables,"A:Y",first_row=2,stop_in_row_function=stop_condition))

Row,Gen/Hour,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,Wind2,31.167,29.4256,27.6723,28.6323,28.7513,29.2233,28.6362,29.739,30.4213,31.766,33.1306,36.4072,38.4619,40.5286,41.3934,42.516,45.4951,42.6112,37.7479,34.4872,35.9827,35.471,32.5038,31.2067
2,Solar8,0.0,0.0,0.0,0.0,0.000148093,7.14521,32.1157,64.6244,84.5466,90.8603,92.6444,93.0311,92.748,91.194,86.7143,73.3265,41.8975,14.7328,0.62214,0.0,0.0,0.0,0.0,0.0


In [ ]:
df4["1"]

Lineas

In [19]:
df5 = DataFrame(XLSX.gettable(lineas,"A:G",first_row=1,stop_in_row_function=stop_condition))

Row,Branch Name,FromBus,ToBus,Resistance (R),Reactance (X),LineCharging (B),Max Flow [MW]
,Any,Any,Any,Any,Any,Any,Any
1,Line1To2,1,2,0.01938,0.05917,0,135
2,Line1To5,1,5,0.05403,0.22304,0,135
3,Line2To3,2,3,0.04699,0.19797,0,135
4,Line2To4,2,4,0.05811,0.17632,0,135
5,Line2To5,2,5,0.05695,0.17388,0,135
6,Line3To4,3,4,0.06701,0.17103,0,135
7,Line4To5,4,5,0.01335,0.04211,0,135
8,Line4To7,4,7,0,0.20912,0,135
9,Line4To9,4,9,0,0.55618,0,135


# Modelo JuMP

In [12]:
### Problema optimizacion
model = Model(Gurobi.Optimizer) # Crear objeto "modelo" con el solver Gurobi

## Variables
I = 20
T = 24
N = 5
# Variables Economic Dispatch
@variable(model, p[1:I, 1:T] >= 0)  # Potencia activa de generador i en tiempo t. Valor en p.u.
@variable(model, d[1:N, 1:T])       # angulo (d de degree) de la barra n en tiempo t

# Variables nuevas (UC)
# No se utiliza potencia reactiva: Aproximacion DC
#@variable(model, q[1:I, 1:T])       # Potencia reactiva de generador i en tiempo t. Valor en p.u. (puede ser < 0)
@variable(model, u[1:I, 1:T], Bin)       # AGREGAR NATURALEZA {0,1}. Indica encendido de gen i en t.
@variable(model, v[1:I, 1:T], Bin)       # AGREGAR NATURALEZA {0,1}. Indica apagado de gen i en t.
@variable(model, w[1:I, 1:T], Bin)       # AGREGAR NATURALEZA {0,1}. Estado ON(1)/OFF(0) de gen i en t.

# Variables BESS (no se utilizan)
#@variable(model, pb[1:B, 1:T])      # flujo de potencia del BESS b en tiempo t
#@variable(model, e[1:B,1:T] >=0)    # energia almacenada en el BESS e en tiempo t

## Funcion Objetivo
#@objective(model, Min, sum(Gen[i].VariableCostCost * p[i,t] +  Gen[i].FixedCost * w[i,t] + Gen[i].StartCost * u[i,t] for i in 1:I, t in 1:T ))

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-26


20×24 Matrix{VariableRef}:
 w[1,1]   w[1,2]   w[1,3]   w[1,4]   w[1,5]   …  w[1,22]   w[1,23]   w[1,24]
 w[2,1]   w[2,2]   w[2,3]   w[2,4]   w[2,5]      w[2,22]   w[2,23]   w[2,24]
 w[3,1]   w[3,2]   w[3,3]   w[3,4]   w[3,5]      w[3,22]   w[3,23]   w[3,24]
 w[4,1]   w[4,2]   w[4,3]   w[4,4]   w[4,5]      w[4,22]   w[4,23]   w[4,24]
 w[5,1]   w[5,2]   w[5,3]   w[5,4]   w[5,5]      w[5,22]   w[5,23]   w[5,24]
 w[6,1]   w[6,2]   w[6,3]   w[6,4]   w[6,5]   …  w[6,22]   w[6,23]   w[6,24]
 w[7,1]   w[7,2]   w[7,3]   w[7,4]   w[7,5]      w[7,22]   w[7,23]   w[7,24]
 w[8,1]   w[8,2]   w[8,3]   w[8,4]   w[8,5]      w[8,22]   w[8,23]   w[8,24]
 w[9,1]   w[9,2]   w[9,3]   w[9,4]   w[9,5]      w[9,22]   w[9,23]   w[9,24]
 w[10,1]  w[10,2]  w[10,3]  w[10,4]  w[10,5]     w[10,22]  w[10,23]  w[10,24]
 w[11,1]  w[11,2]  w[11,3]  w[11,4]  w[11,5]  …  w[11,22]  w[11,23]  w[11,24]
 w[12,1]  w[12,2]  w[12,3]  w[12,4]  w[12,5]     w[12,22]  w[12,23]  w[12,24]
 w[13,1]  w[13,2]  w[13,3]  w[13,4]  w[13,5]  

In [20]:
model.VariableRef

ErrorException: type GenericModel has no field VariableRef

# Modelo

In [12]:
# Tarea 2 Operacion Economica de Sistemas Electricos
# Vicente Goehring & Sebastian Villagra
# Mayo 2024

# Codigo busca crear un modelo de optimizacion para el Unit Commitment
# de forma mas general posible, i.e., entra como input cualquier CSV con
# los datos de un sistema y optimiza de forma robusta.


#Iniciación de paquetes
using DataFrames, CSV
using JuMP , HiGHS      # https://jump.dev/JuMP.jl/stable/manual/models/ 
using Gurobi
using XLSX  # Se agrega para leer archivos .xlsx (Excel)

#Definición de struct: FALTA AGREGAR MAS INSTANCIAS DE CADA STRUCT (seba)  

# Barras
struct Bus
    Id::Int64       # Id del Bus
    Vmax::Float64   # Voltaje maximo [pu]
    Vmin::Float64   # Voltaje minimo [pu]
    Gs::Float64
    Bs::Float64
end

# Generadores
struct Generador
    Name::String
    Bus::Int64    
    PotMax::Float64
    PotMin::Float64
    Qmax::Float64
    Qmin::Float64
    Ramp::Float64
    Sramp::Float64
    MinUp::Float64
    MinDn::Float64
    InitS::Float64
    InitP::Float64
    StartCost::Float64
    FixedCost::Float64
    VariableCost::Float64
    Tech::String
    PminFactor::Float64 
    Qfactor::Float64
    RampFactor::Float64
    StartUpCostFactor::Int64
end

# Potencias demandadas
# La demanda la dejaria como un DataFrame y listo
struct Demanda
    T::Int64
    Barra1::Int64
    Barra2::Int64
    Barra3::Int64
    Barra4::Int64
    Barra5::Int64
    Barra6::Int64
    Barra7::Int64
    Barra8::Int64
    Barra9::Int64
end

# Lineas de transmision
# IMPORTANTE:   case014.xlsx tiene el inicio y fin como string. case118.xlsx los tiene como int
#               Por lo tanto, se modifica el archivo case014.xlsx para poder leer de forma automatica
struct Linea
    Name::String    
    Inicio::Int64       # Barra de Inicio
    Fin::Int64          # Barra de Fin
    R::Float64          # Resistencia
    X::Float64          # Reactancia
    LineCharging::Int64 # (B)
    PotMax::Int64       # Potencia maxima a transportar
end
# Battery Energy Storage System
struct BESS
    Id::Int64
    PotMax::Int64
    Duration::Int64
    Eff::Float64
    Estart::Float64
    Efinal::Float64
    Barra::Int64
end
# Renewables -> tambien dejarlo como DataFrame

##  Archivo Excel a leer
archivo = "Case014.xlsx"    # se comienza con el caso mas pequeño
xf = XLSX.readxlsx(archivo) # leer el archivo excel

# Separar hojas del archivo
sheet_buses = xf["Buses"]
sheet_demandas = xf["Demand"]
sheet_generadores = xf["Generators"]
sheet_lineas = xf["Lines"]
sheet_renovables = xf["Renewables"]

# Definir la función de parada
# Se crea el DataFrame hasta que se reconozca la entrada "END"
# De esta forma se puede leer tablas con cualquier numero de filas
#   siempre y cuando terminen con un "END".
function stop_condition(row)
    return isempty(row[1]) || row[1] == "END"
end

# Crear DataFrames
# XLSX.gettable(objeto hoja excel, Columnas de tabla, first_row= primera fila a considerar, stop_in_row_function: condicion dejar de leer)
buses_ref       = DataFrame(XLSX.gettable(sheet_buses,"A:E",first_row=1,stop_in_row_function=stop_condition))
demandaP_ref    = DataFrame(XLSX.gettable(sheet_demandas,"A:Y",first_row=2,stop_in_row_function=stop_condition))
demandaQ_ref    = DataFrame(XLSX.gettable(sheet_demandas,"AA:AY",first_row=2,stop_in_row_function=stop_condition))
generadores_ref = DataFrame(XLSX.gettable(sheet_generadores,"A:T",first_row=1,stop_in_row_function=stop_condition))
lineas_ref      = DataFrame(XLSX.gettable(sheet_lineas,"A:G",first_row=1,stop_in_row_function=stop_condition))
renovables_ref  = DataFrame(XLSX.gettable(sheet_renovables,"A:Y",first_row=2,stop_in_row_function=stop_condition))
# Notar que se crean 2 DataFrame a partir de la hoja "Demand": Potencia activa (P) y potencia reactiva (Q).
# Ademas, la fila considerada como inicio de la tabla es la fila 2, ya que la fila 1 solo hace distincion entre estas 2 potencias.
# Lo mismo ocurre en el DataFrame "renovables_ref".

# ESTO SE ELIMINARIA
#######################################################
#Importación de archivos
#generators_ref  = CSV.File("Generators.csv") |> DataFrame   # Crear DataFrame generadores
#demand_ref      = CSV.File("Demand.csv") |> DataFrame       # Crear DataFrame demanda
#lines_ref       = CSV.File("Lines.csv") |> DataFrame        # Crear DataFrame lineas tx
#batteries_ref   = CSV.File("Bess.csv") |> DataFrame         # Crear DataFrame baterias
# Crear una copia 
#generators      = copy(generators_ref)
#demand          = copy(demand_ref)
#lines           = copy(lines_ref)
#batteries       = copy(batteries_ref)
#######################################################

# Crear copias (aparentemente buena practica)
buses = copy(buses_ref)
demandP = copy(demandaP_ref[:,2:25])
demandQ = copy(demandaQ_ref)
generators = copy(generadores_ref)
lines = copy(lineas_ref)
renewables = copy(renovables_ref) 

#inclusión en variables
P_base = 100
T = ncol(demandP)           # N° bloques temporales
N = nrow(buses)             # N° nodos 
I = nrow(generators)        # N° Generadores
L = nrow(lines)             # N° Lineas
#B = nrow(batteries)        # N° BESS

# Debugg check
println("Prueba de numero buses, deberia ser 14: ", N)

# HASTA ACA LLEGUE (SEA -quien mas xdxdxdxd): 17/05 -

# Listas que almacenan los Structs 
Buses       = []
Gen         = []
Demandas    = []
Lineas      = []
#Baterias    = []

## Crear los Structs

# Struct Bus
# struct Bus
#     Id::Int64       # Id del Bus
#     Vmax::Float64   # Voltaje maximo [pu]
#     Vmin::Float64   # Voltaje minimo [pu]
#     Gs::Float64
#     Bs::Float64
# end
for i in 1:N
    x = Bus(
        buses[i,1], # 1- Id
        buses[i,2], # 2- Vmax [pu]
        buses[i,3], # 3- Vmin [pu]
        buses[i,4], # 4- Gs
        buses[i,5]) # 8- Bs   
    push!(Buses, x)
end

# Struct Generadores
# struct Gen
#     (1)Id:: Int64              
#     (2)Name::String
#     (3)Bus::Int64    
#     (4)PotMax::Float64
#     (5)PotMin::Float64
#     (6)Qmax::Float64
#     (7)Qmin::Float64
#     (8)Ramp::Int64
#     (9)Sramp::Int64
#     (10)MinUp::Int64
#     (11)MinDn::Int64
#     (12)InitS::Int64
#     (13)InitP::Int64
#     (14)StartCost::Int64
#     (15)FixedCost::Int64
#     (16)VariableCost::Int64
#     (17)Type::String
#     (18)PminFactor::Float64 
#     (19)Qfactor::Float64
#     (20)RampFactor::Float64
#     (21)StartUpCostFactor::Int64
# end

for i in 1:I
    println("Generador", i)
    x = Generador(
        generators[i,1],    # 1- Name
        generators[i,2],    # 2- Bus
        generators[i,3],    # 3- Pmax
        generators[i,4],    # 4- Pmin
        generators[i,5],    # 5- Qmax
        generators[i,6],    # 6- Qmin
        generators[i,7],    # 7- Ramp [Mw/h]
        generators[i,8],    # 8- Sramp [Mw]
        generators[i,9],    # 9- MinUp
        generators[i,10],   # 10- MinDn
        generators[i,11],   # 11- InitS
        generators[i,12],   # 12- InitP
        generators[i,13],   # 13- StartCost
        generators[i,14],   # 14- FixedCost
        generators[i,15],   # 15- VariableCost
        generators[i,16],   # 16- Type
        generators[i,17],   # 17- PminFactor
        generators[i,18],   # 18- Qfactor
        generators[i,19],   # 19- RampFactor
        generators[i,20])   # 20- StartUpCostFactor
    push!(Gen, x)
end
## println(Generadores[1].PotMax) ejemplo de como obtener un prámetro en particular

# Struct Linea
# struct Linea
#     Id::Int64
#     Name::String    
#     Inicio::Int64       # Barra de Inicio
#     Fin::Int64          # Barra de Fin
#     R::Float64          # Resistencia
#     X::Float64          # Reactancia
#     LineCharging::Int64 # (B)
#     PotMax::Int64       # Potencia maxima a transportar
# end
for i in 1:L
    x = Linea(
        lines[i,1], # 1- Name 
        lines[i,2], # 2- StartBus
        lines[i,3], # 3- EndBus
        lines[i,4], # 4- Resistance
        lines[i,5], # 5- Reactance
        lines[i,6], # 6- LineCharging(B)
        lines[i,7]) # 7- MaxFlow [Mw]   
    push!(Lineas, x)
end

println("Demanda nodo 14, hora 2: ", demandP[14,2])
println("Size de demandaP_ref: ", size(demandaP_ref))
println("Size de demandP: ", size(demandP))

# Struct demanda (quizas quitarlo)
# for i in 1:N
#     x = [demand[i,2],demand[i,3],demand[i,4],demand[i,5],demand[i,6],demand[i,7]]
#     push!(Demandas, x)
# end

# Crear Structs BESS
# for b in 1:B
#     x = BESS(batteries[b,1],batteries[b,2],batteries[b,3],batteries[b,4],
#             batteries[b,5],batteries[b,6],batteries[b,7])
#     push!(Baterias, x)
# end


####################################################################
#               MODELO OPTIMIZACION
####################################################################

### Problema optimizacion
model = Model(Gurobi.Optimizer) # Crear objeto "modelo" con el solver Gurobi

## Variables

# Variables Economic Dispatch
@variable(model, p[1:I, 1:T] >= 0)  # Potencia activa de generador i en tiempo t. Valor en p.u.
@variable(model, d[1:N, 1:T])       # angulo (d de degree) de la barra n en tiempo t

# Variables nuevas (UC)
# No se utiliza potencia reactiva: Aproximacion DC
#@variable(model, q[1:I, 1:T])       # Potencia reactiva de generador i en tiempo t. Valor en p.u. (puede ser < 0)
@variable(model, u[1:I, 1:T], Bin)       # AGREGAR NATURALEZA {0,1}. Indica encendido de gen i en t.
@variable(model, v[1:I, 1:T], Bin)       # AGREGAR NATURALEZA {0,1}. Indica apagado de gen i en t.
@variable(model, w[1:I, 1:T], Bin)       # AGREGAR NATURALEZA {0,1}. Estado ON(1)/OFF(0) de gen i en t.

# Variables BESS (no se utilizan)
#@variable(model, pb[1:B, 1:T])      # flujo de potencia del BESS b en tiempo t
#@variable(model, e[1:B,1:T] >=0)    # energia almacenada en el BESS e en tiempo t

## Funcion Objetivo
@objective(model, Min, sum(Gen[i].VariableCost * p[i,t] +  Gen[i].FixedCost * w[i,t] + Gen[i].StartCost * u[i,t] for i in 1:I, t in 1:T ))

### Restricciones

# Equilibrio de Potenica (APROXIMACION: Flujo DC  --> Demanda Potencia Activa)
@constraint(model, DCPowerFlowConstraint[n in 1:N, t in 1:T], 
sum(p[i,t] for i in 1:I if Gen[i].Bus == n) - demandP[n,t]      == 
P_base*sum( (1/Lineas[l].X) * (d[Lineas[l].Inicio,t] - d[Lineas[l].Fin,t]) for l in 1:L if Lineas[l].Inicio == n)
+ P_base*sum( (1/Lineas[l].X) * (d[Lineas[l].Fin,t] - d[Lineas[l].Inicio,t]) for l in 1:L if Lineas[l].Fin == n))

# + sum(pb[b,t] for b in 1:B if Baterias[b].Barra == n) | Al lado izquierdo de la ecuación (baterias)

#Flujo en lineas. Se considera o de origen, y d de destino
@constraint(model, LineMaxPotInicioFin[l in 1:L, t in 1:T], 1/Lineas[l].X * (d[Lineas[l].Inicio,t] - d[Lineas[l].Fin,t]) <= Lineas[l].PotMax/P_base) 
@constraint(model, LineMinPotFinInicio[l in 1:L, t in 1:T], -1/Lineas[l].X * (d[Lineas[l].Inicio,t] - d[Lineas[l].Fin,t]) <= Lineas[l].PotMax/P_base)
#Angulo de referencia
@constraint(model, RefDeg[1, t in 1:T], d[1,t] == 0)  

### Restricciones de generadores

## Potencias Activas y reactivas
# Potencia Activa maxima
@constraint(model, PMaxConstraint[i in 1:I, t in 1:T], p[i,t] <= w[i,t]*Gen[i].PotMax)
# Potencia Activa minima
@constraint(model, PMinConstraint[i in 1:I, t in 1:T], w[i,t]*Gen[i].PotMin <= p[i,t])

# Potencia Reactiva maxima
#@constraint(model, QMaxConstraint[i in 1:I, t in 1:T], q[i,t] <= Gen[i].QMax)
# Potencia Reactiva minima
#@constraint(model, QMinConstraint[i in 1:I, t in 1:T], Gen[i].QMin <= q[i,t])

## Rampas
# Rampa up
@constraint(model, RampUpConstaint[i in 1:I, t in 2:T], (p[i,t] - p[i,t-1]) <= Gen[i].Ramp + Gen[i].Sramp*u[i,t])
# Rampa dn
@constraint(model, RampDownConstaint[i in 1:I, t in 2:T], (p[i,t] - p[i,t-1]) >= 0-Gen[i].Ramp - Gen[i].Sramp*v[i,t])

## Estados Binarios
@constraint(model, BinaryState[i in 1:I, t in 2:T], (u[i,t] - v[i,t]) == (w[i,t] - w[i,t-1]))
#@constraint(model, BinaryInitial[i in 1:I], (w[i,1] = State0[i]))   # Condicion inicial, por ahora libre

## Tiempo minimo de encendido: sumo todos los x dentro de la ventana desde t=1 hasta el instante enterior al encendido
@constraint(model, MinUpTime[i in 1:I, t in 2:T], sum(w[i,k] for k in 1:(t-1) if k >= t-Gen[i].MinUp) >= v[i,t]*Gen[i].MinUp)
## Tiempo minimo de apagado
@constraint(model, MinDnTime[i in 1:I, t in 2:T], sum((1-w[i,k]) for k in 1:(t-1) if k >= t-Gen[i].MinDn) >= u[i,t]*Gen[i].MinDn)



# Restricciones de BESS
# Energia maxima
#@constraint(model, EnergyMax[b in 1:B, t in 1:T], e[b,t] <= (Baterias[b].PotMax)*(Baterias[b].Duration))
# Potencia maxima descarga
#@constraint(model, PMaxBessDischarge[b in 1:B, t in 1:T], pb[b,t] <= Baterias[b].PotMax)
# Potencia maxima carga
#@constraint(model, PMaxBessCharge[b in 1:B, t in 1:T], pb[b,t] >= -1*(Baterias[b].PotMax))
## Imposibilidad de generacion - E inicial
#@constraint(model, Estart[b in 1:B], e[b,1] == (Baterias[b].Estart)*(Baterias[b].PotMax)*(Baterias[b].Duration))
# Imposibilidad de generacion - E final
#@constraint(model, Efinal[b in 1:B], e[b,T] == (Baterias[b].Efinal)*(Baterias[b].PotMax)*(Baterias[b].Duration))
# Dinamica BESS
#@constraint(model, DynamicBESS[b in 1:B, t in 2:T], e[b,t] == e[b,t-1] - pb[b,t])

#############################################################


################################################ ACTUALIZADO 24-05 (00:10 AM) -SEA ################################
### Notas:
#   - Siento que hay que agregar BinaryState0 -> Con los resultados se pueden analizar incongruencias
#   - Por lo mismo, las constraints de MinUpTime y MinDnTime no me terminan de convencer
#   - SI O SI, hay que fijar p[r,t] == gr[t], es decir, las potencias generadas por renovables estan
#       fijas. HINT: los renovables están al final de la tabla de generadores. 

# constraint[i in (I-a):I, t in 1:T], p[i,t] == renewable[i,t] | una wea asi

##############################################################


JuMP.optimize!(model)

Prueba de numero buses, deberia ser 14: 14
Generador1
Generador2
Generador3
Generador4
Generador5
Generador6
Generador7
Demanda nodo 14, hora 2: 12.150115830115833
Size de demandaP_ref: (14, 25)
Size de demandP: (14, 24)
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-26
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2461 rows, 1008 columns and 7056 nonzeros
Model fingerprint: 0x2a83c774
Variable types: 504 continuous, 504 integer (504 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [2e+01, 3e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+02]
Presolve removed 1429 rows and 347 columns
Presolve time: 0.05s
Presolved: 1032 rows, 661 columns, 4054 nonzeros
Variab

┌ Warning: Axis contains one element: 1. If intended, you can safely ignore this warning. To explicitly pass the axis with one element, pass `[1]` instead of `1`.
└ @ JuMP.Containers C:\Users\sevil\.julia\packages\JuMP\OUdu2\src\Containers\DenseAxisArray.jl:186


## Valor objetivo

In [13]:
JuMP.objective_value(model)

180456.9974258979

In [14]:
for t=1:T
    println("Valor en t=", t," :", JuMP.value(p[1,t]))
end

Valor en t=1 :100.0
Valor en t=2 :97.2
Valor en t=3 :94.1
Valor en t=4 :81.00080437572413
Valor en t=5 :74.60080437572412
Valor en t=6 :90.5
Valor en t=7 :100.0
Valor en t=8 :100.0
Valor en t=9 :100.0
Valor en t=10 :100.0
Valor en t=11 :100.0
Valor en t=12 :100.0
Valor en t=13 :100.0
Valor en t=14 :100.0
Valor en t=15 :100.0
Valor en t=16 :100.0
Valor en t=17 :100.0
Valor en t=18 :100.0
Valor en t=19 :100.0
Valor en t=20 :100.0
Valor en t=21 :100.0
Valor en t=22 :100.0
Valor en t=23 :100.0
Valor en t=24 :100.0


In [11]:
ncol(demandP)

24